# Reporte analítica de datos - Proyecto de Grado II

## Preparación de los datos para el municipio de BUGA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from pandas.plotting import lag_plot
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import datetime as datetime

In [ ]:
buga = pd.read_csv('../../dengue_data/Buga.csv')
df = pd.DataFrame(buga)
df["FecNot"]=pd.to_datetime(df['FecNot'])

## Pruebas de hipóstesis para determinar si las series de tiempo son estacionarias | DICKEY-FULLER TEST

**Hipótesis nula:** La serie de tiempo no es estacionaria

**Hipótesis del investigador:** La serie de tiempo es estacionaria

*Si el estadístico ADF < Valor crítico entonces rechazo la hipótesis nula*  
*Si el estadístico ADF > Valor crítico entonces acepto la hipótesis nula*

**Se trabajarán las series de tiempo individuales por barrios entre los años 2013 y 2016**

In [ ]:
df2 = df[df.Ano.isin([2013, 2016])]
data_model = pd.DataFrame(df2['FecNot'].value_counts().sort_index())
data_model.columns = ['Casos']
data_model.index.name = 'Fecha'

In [ ]:
df2

In [ ]:
def stationarity_analysis_nb(nb_df, name):
    results = adfuller(nb_df)
    print('Estadístico ADF: %f' % results[0])
    print('p-value: %f' % results[1])
    print('Valor crítico:')
    for key,value in results[4].items():
        print('\t%s: %.3f'% (key,value))

    if results[0] < results[4]["5%"] :
        print("Rechazo Ho: La serie de tiempo es estacionaria")
        nb_df.plot(kind='line', figsize=(30,5))
        plt.title(name)  
        plt.ylabel("Número de casos")    
    else :
        print("Acepta Ho: La serie de tiempo no es estacionaria\n\nDebido a que la serie de tiempo no es estacionaria se procede a transformarla:\n")
        nb_df = nb_df.diff().dropna()
        results = adfuller(nb_df)
        print('Estadístico ADF: %f' % results[0])
        print('p-value: %f' % results[1])
        print('Valor crítico:')
        for key,value in results[4].items():
            print('\t%s: %.3f'% (key,value))

        if results[0] < results[4]["5%"] :
            print("Rechazo Ho: La serie de tiempo es estacionaria")
        else :
            print("Acepta Ho: La serie de tiempo no es estacionaria")

        nb_df.plot(kind='bar')

In [ ]:
def assign_zeros(nb_df) :
    day_delta = datetime.timedelta(days=1)
    start_date = datetime.date(2013, 1, 1)
    end_date = datetime.date(2017, 1, 1)

    list = []

    for i in range((end_date - start_date).days):
        date = start_date + i*day_delta
        try: 
            list.append(nb_df.at[date,'Casos'])
        except:  
            list.append(0)
            
    final_model = pd.DataFrame(columns=('Fecha', 'Casos'))

    for i in range((end_date - start_date).days):
        date = start_date + i*day_delta
        final_model.loc[len(final_model)]=[date, list[i]]
        
    return final_model

Se completan los registros (1461) entre los años 2010 y 2018. El día donde no haya un acaso se agrega con un 0.

**Después para cada barrio se hace su respectivo análisis de estacionariedad**

In [ ]:
neighborhoods_df = df2['Barrio'].value_counts().sort_index()

for neighborhood_name in neighborhoods_df.index :
    nb_df = df2[df2.Barrio == neighborhood_name]
    h = pd.DataFrame(nb_df['FecNot'].value_counts().sort_index())
    h.columns = ['Casos']
    h.index.name = 'Fecha'
    df_zeros = assign_zeros(h)
    df_zeros.set_index('Fecha',inplace=True)
    stationarity_analysis_nb(df_zeros,neighborhood_name)

**TODAS LAS SERIES DE TIEMPO PARA CADA UNO DE LOS BARRIOS ENTRE 2013 Y 2016 SON ESTACIONARIAS**

## PRUEBAS DE CORRELACIÓN

In [ ]:
lag_plot(data)
plt.show()

In [ ]:
values = DataFrame(data.values)
dataframe = concat([values.shift(1), values], axis=1)
dataframe.columns = ['t-1', 't+1']
result = dataframe.corr()
print(result)

In [ ]:
autocorrelation_plot(data)
plt.show()